In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
df = pd.read_csv('CompleteIndexes.csv', parse_dates=True, index_col=0)
df.head(30)

,sp500_Open,sp500_High,sp500_Low,sp500_Close,sp500_Volume,vix_Open,vix_High,vix_Low,vix_Close,vix_Volume,...,n100_High,n100_Low,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.05,40.634806,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.62,NaN,NaN,NaN,NaN
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,9.318000e+08,24.360001,26.150000,23.980000,24.209999,0.0,...,996.770020,996.770020,996.770020,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1.009000e+09,24.940001,27.180000,24.799999,27.010000,0.0,...,955.969971,955.969971,955.969971,0.0,NaN,0.99,NaN,NaN,NaN,NaN
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1.085500e+09,27.980000,29.000000,25.850000,26.410000,0.0,...,930.260010,930.260010,930.260010,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1.092300e+09,26.680000,26.709999,24.700001,25.730000,0.0,...,922.460022,922.460022,922.460022,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1.225200e+09,25.139999,25.170000,21.719999,21.719999,0.0,...,943.880005,943.880005,943.880005,0.0,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.78,NaN,NaN,NaN,NaN
2000-01-10,1441.469971,1464.359985,1441.469971,1457.599976,1.064800e+09,21.889999,22.490000,21.360001,21.709999,0.0,...,960.219971,960.219971,960.219971,0.0,NaN,0.00,NaN,NaN,NaN,NaN


In [3]:
def imput_moving_average_to_col(df_to_input, num_values):
    for index, row in df_to_input.iterrows():
        if pd.isna(row[0]):
            #print("_______________", row)
            count_prev = 0
            mean = 0
            index_prev = df_to_input.index.get_loc(index)
            while count_prev < num_values:
                index_prev -= 1
                if index_prev < 0:
                    break
                if not pd.isna(df_to_input.iloc[index_prev][0]):
                    count_prev += 1
                    #print(df_to_input.iloc[index_prev])
                    mean += df_to_input.iloc[index_prev]
            count_next = 0
            index_next = df_to_input.index.get_loc(index)
            while count_next < num_values:
                index_next += 1
                if index_next >= len(df_to_input):
                    break
                if not pd.isna(df_to_input.iloc[index_next][0]):
                    count_next += 1
                    #print(df_to_input.iloc[index_next])
                    mean += df_to_input.iloc[index_next]
            mean = mean / (count_prev+count_next)
            #print("MEAN" , mean, "----------------")
            df_to_input.at[index, df_to_input.columns[0]]  = mean
    return df_to_input

In [4]:
for col in df.columns:
    df_imp = imput_moving_average_to_col(df[[col]],3)
    df[col] = df_imp
    print("Column", col, "was imputed.")
df.head()

Column sp500_Open was imputed.
Column sp500_High was imputed.
Column sp500_Low was imputed.
Column sp500_Close was imputed.
Column sp500_Volume was imputed.
Column vix_Open was imputed.
Column vix_High was imputed.
Column vix_Low was imputed.
Column vix_Close was imputed.
Column vix_Volume was imputed.
Column dji_Open was imputed.
Column dji_High was imputed.
Column dji_Low was imputed.
Column dji_Close was imputed.
Column dji_Volume was imputed.
Column ndx_Open was imputed.
Column ndx_High was imputed.
Column ndx_Low was imputed.
Column ndx_Close was imputed.
Column ndx_Volume was imputed.
Column n255_Open was imputed.
Column n255_High was imputed.
Column n255_Low was imputed.
Column n255_Close was imputed.
Column n255_Volume was imputed.
Column ftse_Open was imputed.
Column ftse_High was imputed.
Column ftse_Low was imputed.
Column ftse_Close was imputed.
Column ftse_Volume was imputed.
Column hsi_Open was imputed.
Column hsi_High was imputed.
Column hsi_Low was imputed.
Column hsi_C

,sp500_Open,sp500_High,sp500_Low,sp500_Close,sp500_Volume,vix_Open,vix_High,vix_Low,vix_Close,vix_Volume,...,n100_High,n100_Low,n100_Close,n100_Volume,Overall EMV Tracker,infectious_daily_infect_emv_index,GPR,trade_US Trade Policy Uncertainty,trade_Japanese Trade Policy Uncertainty,trade_Trade Policy EMV Fraction
2000-01-01,1441.296672,1448.829997,1404.490031,1418.916667,1.008767e+09,25.760001,27.443333,24.876666,25.876666,0.0,...,961.000000,961.000000,961.000000,0.0,23.477003,1.05,40.634806,70.148805,69.294997,0.029394
2000-01-02,1441.296672,1448.829997,1404.490031,1418.916667,1.008767e+09,25.760001,27.443333,24.876666,25.876666,0.0,...,961.000000,961.000000,961.000000,0.0,23.477003,2.62,41.223446,70.148805,69.294997,0.029394
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,9.318000e+08,24.360001,26.150000,23.980000,24.209999,0.0,...,996.770020,996.770020,996.770020,0.0,23.477003,0.00,41.223446,70.148805,69.294997,0.029394
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1.009000e+09,24.940001,27.180000,24.799999,27.010000,0.0,...,955.969971,955.969971,955.969971,0.0,23.477003,0.99,41.223446,70.148805,69.294997,0.029394
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1.085500e+09,27.980000,29.000000,25.850000,26.410000,0.0,...,930.260010,930.260010,930.260010,0.0,23.477003,0.00,41.321553,70.148805,69.294997,0.029394


In [5]:
df.to_csv(r'MovingAverageImputed.csv', index = True)